# Stacking Notebook for HOYS LCO images

Welcome to the stacking notebook for the HOYS LCO Project.\
Please see the video tutorial and upload your images to the 'upload_raw_images_here' folder.
Best to upload the compressed images one by one since they are large and may take some time to upload.

### Notebook tips:
* ***Shift + Enter on a code cell/block to run it and advance to the next cell.***
* Can re-run blocks out of order as long as the variables are already there
* If python kernal crashses or out of memory, use the reset button above to reset the kernal.

In [ ]:
!pip install MontagePy

In [ ]:
import os
import sys
import numpy as np
import astropy.io.fits as pyfits
from astropy.coordinates import ICRS
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import SkyCoord

from scipy import stats
from scipy import optimize #Leastsq Levenberg-Marquadt Algorithm
from scipy.interpolate import UnivariateSpline
from scipy import interpolate
#for plotting
import matplotlib
import matplotlib.pyplot as plt
from scipy.signal import medfilt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
from IPython.display import Image
import MontagePy.main as mon

* Enter username to append to final stacked images filename, please do not use spaces
* *Do not re run code blocks with text entry or selection boxes, this will reset the box to default value*
* *The value entereed or selected in the boxes will automatically update once shown*

In [ ]:
user=widget.Text(
    value='username',
    description='Enter username (no spaces):',
    layout={'width': '300px'},style = {'description_width': 'initial'}
)
display(user)

* Set working directories and list raw files uploaded.
* If no files listed, please check/re-upload images

In [ ]:
#if running on google colab, create directories
os.mkdir('final_stacked')
os.mkdir('filters')
os.mkdir('upload_raw_images_here')

In [ ]:
raw_images = 'upload_raw_images_here'
pathout = 'final_stacked'
filters='filters'
username=user.value
rep_char='. '
for char in rep_char:
    username=username.replace(char,'')
if username == 'username':
    print('!!! please set username in textbox above and rerun this block!!!')

myfilelist = [ ff for ff in np.sort(os.listdir(raw_images)) if '.fits' in ff ]
print('list of raw images in upload folder:')
myfilelist

* Clean working directories and move raw images to folder by filter

In [ ]:
#clean out the to-add directories
os.system('rm -rf %s/*' % filters)
filtlist=[]
objlist=[]
#loop over all files and sort them into the correct to-add directories
for f in myfilelist:
        #read in the files
        inhdulist = pyfits.open(raw_images+'/'+f)
        #get the filter name and create directory
        try:
            fil = inhdulist[0].header['FILTER']
            objectname = inhdulist[0].header['OBJECT']
            hdu_idx=0
        except: #depending on the fits file is compressed or older, primary index may be different
            fil = inhdulist[1].header['FILTER']
            objectname = inhdulist[1].header['OBJECT']
            hdu_idx=1
        objectname=objectname.replace(' ','')
        filtlist.append(fil)
        objlist.append(objectname)
        filter_folder=os.path.join(filters,fil)
        if os.path.exists(filter_folder) == False:
            os.mkdir(filter_folder)
            print('creating directory for filter:',fil)
        #save the files in pathout directory
        pyfits.writeto(filter_folder+"/"+f.strip('.fz'), inhdulist[hdu_idx].data, inhdulist[hdu_idx].header)

* List filters of raw data and check that all images from same object

In [ ]:
try:
    os.remove('filters/.DS_Store')#remove temp file
except:
    pass
print('List of filters in raw data:')
print(list(np.unique(filtlist)))
print('for object:',list(np.unique(objlist)))
if len(np.unique(objlist)) >1:
    print('wanring: more than one object name detected, check if different objects!')

* Function to stack the images using the python montage wrapper

In [ ]:
def stack_images(pathlist, fil, pathout, username, objectname):
    # For each filter in the pathlist
    print('---stacking images for filter:', fil)
    add_dir = os.path.join(pathlist, fil)
    # Make list of the files in the directory
    myfilelist = [fff for fff in np.sort(os.listdir(add_dir)) if '.fits' in fff]

    os.system('rm -rf montage_rot')
    os.system('rm -rf montage_diff')
    os.system('rm -rf montage_corr')
    os.system('mkdir montage_rot')
    os.system('mkdir montage_diff')
    os.system('mkdir montage_corr')

    print('getting fits header')
    mon.mGetHdr(add_dir + '/' + myfilelist[1], "master.hdr")
    print('creating image table for images to stack')
    mon.mImgtbl(add_dir, "ima.tbl")
    print('reprojecting images')
    mon.mProjExec(add_dir, "ima.tbl", "master.hdr", projdir="montage_rot", debug=1)
    print('creating image table for reprojected images')
    mon.mImgtbl("montage_rot", "images.tbl")
    mon.mOverlaps("images.tbl", "diffs.tbl")
    print('determining overlaps')
    mon.mDiffExec("montage_rot", "diffs.tbl", "master.hdr", "montage_diff")

    print('fitting overlaps')    
    mon.mFitExec("diffs.tbl", "fits.tbl", "montage_diff")
    
    # The missing fourth argument is fit_options, which can be an empty string if no specific options are needed
    print('determining background model')    
    mon.mBgModel("images.tbl", "fits.tbl", "corrections.tbl", "")
    print('fitting background model')
    mon.mBgExec("montage_rot", "images.tbl", "corrections.tbl", "montage_corr")
    print('stacking images')
    mon.mAdd("montage_corr", "images.tbl", "master.hdr", "test.fits", haveAreas=True, coadd=1)

    os.system('rm -rf montage_rot')
    os.system('rm -rf montage_diff')
    os.system('rm -rf montage_corr')
    os.system('rm ima.tbl images.tbl diffs.tbl fits.tbl corrections.tbl master.hdr')
    print('removed temp files')

    sum_exptime = 0.0
    sum_mjd = 0.0
    for ffff in myfilelist:
        data = pyfits.open(add_dir + '/' + ffff)
        try:
            sum_exptime += data[0].header['EXPTIME']
            sum_mjd += data[0].header['MJD-OBS']
        except:
            sum_exptime += data[1].header['EXPTIME']
            sum_mjd += data[1].header['MJD-OBS']
    
    # Calculate the total exposure time and average MJD
    exptime = sum_exptime
    mjd = sum_mjd / len(myfilelist)
    
    # Read in the stacked file and change the header entries
    data, header = pyfits.getdata('test.fits', header=True)
    header['EXPTIME'] = exptime
    header['MJD-OBS'] = mjd
    
    # Write out the changed file
    outfilename = os.path.join(pathout, f'lco_{objectname}_{fil}_{username}.fits')
    pyfits.writeto(outfilename, data, header, overwrite=True)
    
    # Clean up the directory
    os.system("rm test.fits *_area.fits")
    print('***finished stacking images***')


* Check which filter is selected from dropdown menu

In [ ]:
fil_select=widget.Dropdown(
    options=list(np.unique(filtlist)),
    description='Select Filter:'
)
display(fil_select)

* Stack images for selected filter
* This may take a some time to produce an output

In [ ]:
stack_images(filters,fil_select.value,pathout,username,objectname)

* Display image of final stacked fits files

In [ ]:
import os
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Function to display the image with specified gamma and vmax
def display_image(gamma=0.3, vmax_percentile=99.999):
    # Update the data to be visualized based on the selected image
    fits_file = os.path.join(pathout, selected_image.value)
    
    # Open the FITS file
    hdulist = fits.open(fits_file)
    
    # Get the data from the primary HDU (Header/Data Unit)
    image_data = hdulist[0].data
    
    # Close the FITS file after loading data
    hdulist.close()

    # Replace NaNs with zero
    image_data = np.nan_to_num(image_data, nan=0.0)

    # Calculate the minimum positive value in the data
    positive_values = image_data[image_data > 0]
    if len(positive_values) > 0:
        min_val = np.min(positive_values)
    else:
        min_val = 0.1  # or some other small positive value

    # Set vmax to the specified percentile of the data
    vmax = np.percentile(image_data, vmax_percentile)
    
    # Display the image data with matplotlib using PowerNorm
    plt.figure(figsize=(10,8))
    plt.imshow(image_data, cmap='gray', origin='lower', norm=PowerNorm(gamma=gamma, vmin=min_val, vmax=vmax))
    plt.colorbar()
    plt.title(f'FITS Image with Power-Law Stretch (gamma={gamma}, vmax_percentile={vmax_percentile})')
    plt.xlabel('X Pixel')
    plt.ylabel('Y Pixel')
    plt.show()

# List of FITS files in the directory
fits_files = [f for f in os.listdir(pathout) if f.endswith('.fits')]

# Widgets for selecting image and adjusting gamma and vmax
selected_image = widgets.Dropdown(options=fits_files, description='FITS Image:')
gamma_slider = widgets.FloatSlider(value=0.3, min=0.01, max=1, step=0.01, description='Gamma:', continuous_update=False)
vmax_slider = widgets.FloatSlider(value=99.999, min=50.0, max=100.0, step=0.1, description='vmax percentile:', continuous_update=False)

# Display widgets and interactive plot
ui = widgets.VBox([selected_image, gamma_slider, vmax_slider])
out = widgets.interactive_output(display_image, {'gamma': gamma_slider, 'vmax_percentile': vmax_slider})

display(ui, out)
